# Rail network analysis

In [1]:
import collections
import dataclasses
import itertools
import json
import typing

from dataclasses import dataclass

The following sets contain all known Mittelzentren and Oberzentren. We will annotate stations and train lines with this information.

In [2]:
oberzentren = {"Dresden", "Leipzig", "Chemnitz", "Plauen", "Zwickau", "Bautzen", "Görlitz", "Hoyerswerda"}
mittelzentren = {"Annaberg-Buchholz", "Borna", "Coswig", "Crimmitschau", "Delitzsch", "Dippoldiswalde", "Döbeln",
                 "Eilenburg", "Freiberg","Freital", "Glauchau", "Grimma", "Großenhain", "Kamenz", "Limbach-Oberfrohna",
                 "Löbau", "Marienberg", "Markkleeberg", "Meißen", "Mittweida", "Niesky", "Oelsnitz", "Oschatz", "Pirna",
                 "Radeberg", "Radebeul", "Reichenbach", "Riesa", "Schkeuditz", "Stollberg", "Torgau", "Weißwasser",
                 "Werdau", "Wurzen", "Zittau", "Auerbach", "Ellefeld", "Falkenstein", "Rodewisch", "Hohenstein-Ernstthal",
                 "Lichtenstein", "Oberlungwitz", "Aue", "Bad Schlema", "Lauter-Bernsbach", "Lößnitz", "Schneeberg",
                 "Schwarzenberg"}

Load in our data

In [3]:
trainlines_file = open("timetables/timetables.json", "r")
trainlines = json.load(trainlines_file)["data"]
trainlines_file.close()
trainlines[0]

{'train_type': 'VBG',
 'extracted_from': 'Adorf(Vogtl)',
 'line': 'RB4',
 'route_normalized': ['Adorf(Vogtl)',
  'Hundsgrün',
  'Oelsnitz(Vogtl)',
  'Pirk',
  'Weischlitz',
  'Kürbitz',
  'Plauen(Vogtl) Mitte',
  'Barthmühle',
  'Rentzschmühle',
  'Elsterberg-Kunstseidenwerk',
  'Elsterberg',
  'Greiz-Dölau',
  'Greiz',
  'Neumühle(Elster)',
  'Berga(Elster)',
  'Wünschendorf',
  'Gera-Zwötzen',
  'Gera Süd',
  'Gera Hbf']}

We have to capture different train lines in some canonical form to allow for convenient access and ensure comparability. This is exactly, what `line_name` does for us. In addition, it also allows us to generate unique identifiers for those lines, which did not get one in the Bahn API.

In [4]:
train_number_generator = collections.defaultdict(int) # int() returns 0

def line_name(line_data, generate=True):
    if line_data["line"] is None and generate:
        train_number_generator[line_data["train_type"]] += 1
        generated_number = train_number_generator[line_data["train_type"]]
        line_data["line"] = str(generated_number)
    elif line_data["line"] is None:
        return line_data["train_type"]
    elif line_data["line"] == "SEV" and generate:
        train_number_generator["SEV"] += 1
        generated_number = train_number_generator["SEV"]
        line_data["line"] = "SEV" + str(generated_number)
    
    if any(line_data["line"].startswith(prefix) for prefix in ["S", "RE", "RB", "C"]):
        return line_data["line"]

    return line_data["train_type"] + line_data["line"]

In [5]:
@dataclass
class Trainstation:
    name: str
    oberzentrum: bool = False
    mittelzentrum: bool = False
    changes_to_oberzentrum: int = 0 
    changes_to_mittelzentrum: int = 0
    connected_to_oberzentrum: bool = False
    connected_to_mittelzentrum: bool = False
    
        
    def __hash__(self):
        return hash(self.name)
    
    def __eq__(self, other):
        return self.name == other.name

Next, we will build up two data structures at once:
- first up, we create a set of all train stations and annotated each station by whether it is a Mittelzentrum, or even an Oberzentrum
- secondly, we store each train line in a more convenient data structure which enables access to a certain line directly through its name. At the same time we annotate each line by whether it passes a Mittelzentrum, or even an Oberzentrum

In [6]:
stations = {}
lines = {}

for train in trainlines:  
    has_mittelzentrum_stop = False # stores whether the line passed at least one Mittelzentrum
    has_oberzentrum_stop = False # stores whether the line passed at least one Oberzentrum
    
    for stop in train["route_normalized"]:
        station = Trainstation(stop)
        
        # check if the current stop is a known Mittelzentrum or Oberzentrum
        mittelzentrum_stop = any(stop.startswith(mz) for mz in mittelzentren)
        oberzentrum_stop = any(stop.startswith(oz) for oz in oberzentren)
        
        station.mittelzentrum = mittelzentrum_stop or oberzentrum_stop
        station.oberzentrum = oberzentrum_stop
        station.connected_to_mittelzentrum = station.mittelzentrum
        station.connected_to_oberzentrum = station.oberzentrum
        
        # update our train line information
        has_mittelzentrum_stop = has_mittelzentrum_stop or mittelzentrum_stop
        has_oberzentrum_stop = has_oberzentrum_stop or oberzentrum_stop
        
        stations[stop] = station
    
    # store the extended line/route information
    train["name_normalized"] = line_name(train, generate=True)
    train["mittelzentrum_stop"] = has_mittelzentrum_stop or has_oberzentrum_stop
    train["oberzentrum_stop"] = has_oberzentrum_stop
    lines[train["name_normalized"]] = train

To enable quick and convenient access to connections between different stations, we store the whole train line network in a graph structure. Each edge will be annotated by the train lines passing through the edge.

In [7]:
Connection = collections.namedtuple("Connection", ["station", "line"])

@dataclass
class Graph:
    nodes: typing.Set[str] = dataclasses.field(default_factory=set)
    edges: typing.Dict[str, typing.Set[Connection]] = dataclasses.field(default_factory=lambda: collections.defaultdict(set))
        
    def add_edge(self, a, b, line):
        if not a in self.nodes:
            self.nodes.add(a)
        if not b in self.nodes:
            self.nodes.add(b)
            
        self.edges[a].add(Connection(b, line))
        self.edges[b].add(Connection(a, line))

Build the train line graph

In [8]:
network = Graph()

for train in trainlines:   
    # We contruct the graph using a sliding window through the routes. This enables us to capture all local connections.
    window_size = 2
    last_window_idx = len(train["route_normalized"]) - window_size + 1
    for stop_idx in range(last_window_idx):
        connection = train["route_normalized"][stop_idx : stop_idx+window_size]
        start, stop = connection
        network.add_edge(start, stop, line_name(train))

In [9]:
[n for n in network.nodes if n.startswith("Dresden")][:10]

['Dresden-Stetzsch',
 'Dresden-Reick',
 'Dresden-Trachau',
 'Dresden Flughafen',
 'Dresden Hbf (Strehlener Str.)',
 'Dresden-Stetzsch Am Urnenfeld',
 'Dresden Grenzstraße',
 'Dresden-Strehlen',
 'Dresden Industriegelände',
 'Dresden-Kemnitz Flensburger Straße']

In [10]:
list(network.edges["Dresden Hbf"])[:5]

[Connection(station='Dresden-Friedrichstadt', line='RB31'),
 Connection(station='Dresden Mitte', line='S1'),
 Connection(station='Dresden-Neustadt', line='RJ2'),
 Connection(station='Bad Schandau', line='EC2'),
 Connection(station='Dresden Mitte', line='RE15')]

Now we have all the data we need: for each station we know whether it is an Oberzentrum (or Mittelzentrum) or not. For each train route, we know whether it passes through one of these stations. And for each station we know where we can go from there. This enables us, to calculate how many stops we need to reach the next Oberzentrum (Mittelzentrum) for each station in an efficient manner.

But, in order to do so, we build another, more elaborated data structure: the train line graph.
This graph however, is not the canonical modelling of train lines (i.e. stations become nodes and edges indicate a connection). Instead, the train lines themselves become the nodes of the graph and an edge from line A to line B indicates that they share a common station, i.e. it is possible to change from one route to the other.

In [11]:
@dataclass
class LineInfo:
    line: str
    connected_to_oberzentrum: bool = False
    changes_to_oberzentrum: bool = -1

@dataclass
class LineGraph:
    nodes: typing.Dict[str, LineInfo] = dataclasses.field(default_factory=dict)
    edges: typing.Dict[str, typing.Set[str]] = dataclasses.field(default_factory=lambda: collections.defaultdict(set))
        
    def add_edge(self, a, b):
        if not a in self.nodes:
            self.nodes[a] = LineInfo(a)
        if not b in self.nodes:
            self.nodes[b] = LineInfo(b)
            
        self.edges[a].add(b)
        self.edges[b].add(a)
        
        
    def __getitem__(self, key):
        return self.nodes[key]

line_graph = LineGraph()

This line graph will now be filled with our network data. In the end, the graph will allow us to find out how many stops we need for each station quite conveniently.

In [12]:
for station in stations.values():
    connections = network.edges[station.name]
    for intersect in itertools.combinations(connections, 2):
        line_a, line_b = intersect
        line_graph.add_edge(line_a.line, line_b.line)

We start our analysis with exactly those stations, that happen to pass an Oberzentrum.

In [13]:
oberzentrum_stations = [s for s in stations.values() if s.oberzentrum]
oberzentrum_lines = set()
for station in oberzentrum_stations:
    oberzentrum_lines |= set(line.line for line in network.edges[station.name])
oberzentrum_lines

{'C11',
 'C13',
 'C14',
 'C15',
 'EC1',
 'EC2',
 'FLX35',
 'IC1',
 'IC10',
 'IC11',
 'IC12',
 'IC13',
 'IC14',
 'IC15',
 'IC16',
 'IC17',
 'IC18',
 'IC19',
 'IC2',
 'IC20',
 'IC21',
 'IC22',
 'IC23',
 'IC3',
 'IC4',
 'IC5',
 'IC6',
 'IC7',
 'IC8',
 'IC9',
 'ICE1',
 'ICE10',
 'ICE11',
 'ICE12',
 'ICE13',
 'ICE14',
 'ICE15',
 'ICE16',
 'ICE17',
 'ICE18',
 'ICE19',
 'ICE2',
 'ICE20',
 'ICE21',
 'ICE22',
 'ICE23',
 'ICE3',
 'ICE4',
 'ICE5',
 'ICE6',
 'ICE7',
 'ICE8',
 'ICE9',
 'KD1',
 'KD2',
 'KD3',
 'R1',
 'RB1',
 'RB110',
 'RB113',
 'RB13',
 'RB2',
 'RB20',
 'RB22',
 'RB30',
 'RB31',
 'RB33',
 'RB34',
 'RB4',
 'RB45',
 'RB5',
 'RB60',
 'RB61',
 'RB64',
 'RB65',
 'RB80',
 'RB81',
 'RB95',
 'RE1',
 'RE10',
 'RE12',
 'RE13',
 'RE15',
 'RE17',
 'RE18',
 'RE2',
 'RE3',
 'RE42',
 'RE50',
 'RE6',
 'RJ1',
 'RJ2',
 'S1',
 'S2',
 'S3',
 'S4',
 'S5',
 'S5X',
 'S6',
 'SEV1',
 'SEV10',
 'SEV11',
 'SEV2',
 'SEV21',
 'SEV22',
 'SEV23',
 'SEV4',
 'SEV5',
 'SEV6',
 'SEV7',
 'SEV8'}

Now that we now the "good lines", we are ready to calculate the number of changes required for all other lines, using a breath-first style enumeration. To do so, we need to calculate the initial frontier, i.e. the lines which are directly available to us once we sit in a "good line".

In [14]:
oberzentrum_connections = set()
for line in oberzentrum_lines:
    line_graph[line].connected_to_oberzentrum = True
    line_graph[line].changes_to_oberzentrum = 0
    oberzentrum_connections |= line_graph.edges[line]
oberzentrum_connections -= oberzentrum_lines
oberzentrum_connections

{'BusU28',
 'CB37',
 'CB92',
 'RB71',
 'RB72',
 'RBU28',
 'S9',
 'SDG1',
 'SDG2',
 'SDG3',
 'SDG4',
 'SDG5',
 'SEV12',
 'SEV13',
 'SEV14',
 'SEV15',
 'SEV16',
 'SEV17',
 'SEV18',
 'SEV19',
 'SEV20',
 'SEV25',
 'SEV3',
 'SEV9',
 'SOE1',
 'SOE2',
 'TLL7'}

Frontier and search space are set, so we may now finally start the iterations.

In [15]:
# connected lines are the lines we have already seen and therefore do not need to check again
connected_lines = oberzentrum_lines.copy()

# directly adjacent lines to the good lines become the first frontier
frontier = oberzentrum_connections

# we need to change at least once to reach a good line
changes = 1

while frontier:
    next_frontier = set()
    
    for line in frontier:
        line_graph[line].connected_to_oberzentrum = True
        line_graph[line].changes_to_oberzentrum = changes
        next_frontier |= line_graph.edges[line]
    
    connected_lines |= frontier
    next_frontier -= connected_lines
    
    changes += 1
    frontier = next_frontier

And this is our result set:

In [16]:
line_graph.nodes

{'RB2': LineInfo(line='RB2', connected_to_oberzentrum=True, changes_to_oberzentrum=0),
 'RB4': LineInfo(line='RB4', connected_to_oberzentrum=True, changes_to_oberzentrum=0),
 'RB22': LineInfo(line='RB22', connected_to_oberzentrum=True, changes_to_oberzentrum=0),
 'RB13': LineInfo(line='RB13', connected_to_oberzentrum=True, changes_to_oberzentrum=0),
 'RE13': LineInfo(line='RE13', connected_to_oberzentrum=True, changes_to_oberzentrum=0),
 'RE12': LineInfo(line='RE12', connected_to_oberzentrum=True, changes_to_oberzentrum=0),
 'RE1': LineInfo(line='RE1', connected_to_oberzentrum=True, changes_to_oberzentrum=0),
 'RB5': LineInfo(line='RB5', connected_to_oberzentrum=True, changes_to_oberzentrum=0),
 'RE3': LineInfo(line='RE3', connected_to_oberzentrum=True, changes_to_oberzentrum=0),
 'SEV10': LineInfo(line='SEV10', connected_to_oberzentrum=True, changes_to_oberzentrum=0),
 'SEV23': LineInfo(line='SEV23', connected_to_oberzentrum=True, changes_to_oberzentrum=0),
 'S5X': LineInfo(line='S5X'